In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt

%matplotlib notebook

%load_ext dotenv
%dotenv

2.1 Suggest dates
1. Is it a useful feature for our landlords?

First of all, let's see how many request entries are available in the database.
SELECT count(*)
FROM booking
R: 10000 entries

In order to evaluate how useful is the new feature, let's try find when it was created.
SELECT new_move_in_created_at
FROM booking
WHERE new_move_in_at IS NOT NULL
ORDER BY new_move_in_created_at ASC
LIMIT 5
R:
2017-01-02 12:42:54
2017-01-17 16:24:16
2017-03-10 11:12:40
2017-03-29 15:26:18
2017-04-13 16:51:30

We see that this feature was first used in '2017-01-02 12:42:54' but it seems to be almost ignored by landlords. Taking this into account, let's find how many times and when the feature was used.
SELECT month(new_move_in_created_at) month, count(*) count
FROM booking
WHERE new_move_in_at IS NOT NULL
GROUP BY month(new_move_in_created_at);
R:
month count
1	2
3	2
4	3
5	2
6	3
7	133
8	180

With the query above we can see the feature was mostly used (new date requests) in July and August. Since a peak in requests happened in July, we will say that this was the month when the feature was deployed. 

SELECT day(new_move_in_created_at) day, count(*) count
FROM booking
WHERE month(new_move_in_created_at) ='7' AND new_move_in_at IS NOT NULL
GROUP BY day(new_move_in_created_at);

The result of the previous query return a continuous flow in requests since  July 3rd so we will assume this is the date the feature was deployed.

This being said, let's see how many total requests were made in this period of time (since July 3rd).
SELECT count(*)
FROM booking
WHERE requested_at >= '2017-07-03';
R: 8665 entries (out of 10000)

SELECT count(*)
FROM booking
WHERE new_move_in_at IS NULL AND requested_at >= '2017-07-03'
R: 8353 entries not using the new feature

Querying how many times "new_move_in_at" feature was used since this date the result is,
SELECT count(new_move_in_at)
FROM booking
WHERE new_move_in_at IS NOT NULL AND requested_at >= '2017-07-03';
R: 312 entries

We can say that out of 8665 total entries, in a period of 2 months, from beginning of July 2017 until the end of August 2017, "new_move_in_at" was used 312 times (3.6% of entries).

Going into more detail, we will analyse out of those 312 entries, how many resulted in a positive outcome (booking) or negative outcome (no booking).

First let's find which current_state possibilities are available.
SELECT distinct current_state
FROM booking;
R:
expired
paid
rejected
awaiting
payment-expired
confirmed
discarded
cancelled
accepted
requested

A negative outcome will be considered if current_state equal to 'expired', 'rejected', 'discarded' or 'cancelled'. Note that current_state = 'expired' only occurs when the new feature is not used but it will be included for comparison purposes. Therefore,
SELECT COUNT(*)
FROM booking
WHERE new_move_in_at IS NOT NULL AND requested_at >= '2017-07-03'
AND current_state IN('expired', 'rejected', 'discarded', 'cancelled');
R: 117 entries

Total cases of a negative outcome (no booking) without using the new feature are,
SELECT COUNT(*)
FROM booking
WHERE new_move_in_at IS NULL AND requested_at >= '2017-07-03'
AND current_state IN('expired', 'rejected', 'discarded', 'cancelled');
R: 6721 entries

On the other hand, if current_state equal to 'paid', 'payment-expired', 'confirmed' or 'accepted' a booking was made.
SELECT COUNT(*)
FROM booking
WHERE new_move_in_at IS NOT NULL AND requested_at >= '2017-07-03'
AND current_state IN('paid', 'payment-expired', 'confirmed', 'accepted');
R: 195 entries

While total cases of a positive outcome (booking) without using the new feature are,
SELECT COUNT(*)
FROM booking
WHERE new_move_in_at IS NULL AND requested_at >= '2017-07-03' 
AND current_state IN ('paid', 'payment-expired', 'confirmed', 'accepted'); 
R: 1366 entries

From 3rd of July, we are able to say that the probability of a negative outcome (no booking) without using the new feature is about 80.5% (6721/8353), while using the new feature the no booking rate is around 37.5% (117/312). 
From 313 entries of new_move_in_at, 62.5% resulted in a booking (195/312) whereas without the use of new_move_in_at, the probability of a booking is approximately 16.4% (1366/8353).

Concluding, this feature represent a significant increase in successful bookings (from 16.4% to 62.5%), although it is not yet widely used. This featurewhich gives landlords an opportunity to adjust rental slots to efficiently accommodate more students, improving booking rates. 


2. Is there any pattern in the suggested dates? Could this be used to improve the feature, or create new ones?

Let's check how many distinct (unique) dates we can find from landlord suggestions.

Move in dates:
SELECT month(new_move_in_at) month, count(*) count
FROM booking
WHERE new_move_in_at IS NOT NULL AND requested_at >= '2017-07-03'
GROUP BY month(new_move_in_at);
R:
month count
1	2
2	1
3	1
7	11
8	84
9	164
10	44
11	4
12	1

SELECT COUNT(DISTINCT new_move_in_at) as dates
FROM booking
WHERE new_move_in_at IS NOT NULL AND requested_at >= '2017-07-03';
R: 74
74 distinct dates out of 312 entries

As seen above, September (164 entries) and August (84 entries) are the months with more move in suggestions by landlords, while 82 of those dates are unique. 

Move out dates:
SELECT month(new_move_out_at) month, count(*) count
FROM booking
WHERE new_move_out_at IS NOT NULL AND requested_at >= '2017-07-03'
GROUP BY month(new_move_out_at);
R:
month date
1	124
2	25
3	8
4	4
5	6
6	34
7	19
8	26
9	11
10	7
11	10
12	35

SELECT COUNT(DISTINCT new_move_out_at) as dates
FROM booking
R: 98
98 distinct dates out of 312 entries

Regarding move out dates, January (124 entries), December (35 entries) and June (34 entries) are the months with more move out suggestions by landlords, while 98 of those dates are unique. 

From these results, we can go into more detail analysing which dates were most suggested.

Move in dates:
SELECT new_move_in_at, COUNT(new_move_in_at) as date_occurrence
FROM booking
WHERE requested_at >= '2017-07-03'
GROUP BY new_move_in_at
ORDER BY date_occurrence DESC
LIMIT 10
R:
	new_move_in_at	date_occurrence
	2017-09-01 00:00:00	100
	2017-10-01 00:00:00	29
	2017-08-01 00:00:00	20
	2017-08-31 00:00:00	16
	2017-09-16 00:00:00	8
	2017-09-15 00:00:00	6
	2017-09-03 00:00:00	6
	2017-09-05 00:00:00	5
	2017-09-02 00:00:00	5
	2017-08-16 00:00:00	5

Move out dates:
SELECT new_move_out_at, COUNT(new_move_out_at) as date_occurrence
FROM booking
WHERE requested_at >= '2017-07-03'
GROUP BY new_move_out_at
ORDER BY date_occurrence DESC
LIMIT 10;
R:
	new_move_out_at	date_occurrence
	2018-01-31 00:00:00	81
	2018-06-30 00:00:00	23
	2018-08-31 00:00:00	16
	2018-07-31 00:00:00	12
	2018-02-28 00:00:00	10
	2017-12-31 00:00:00	8
	2017-12-30 00:00:00	7
	2018-01-02 00:00:00	6
	2017-11-30 00:00:00	6
	2018-01-17 00:00:00	6

Analysing the results, we can see a pattern happening. In move in dates, the 1st day of the month is the most suggested date, occurring 149 times in the top 3 (August, September and October). On the other hand, in move out dates, the end of the month (e.g. January, June and August) is the priority for landlords.

Taking the point of view of a student and after seeing the results, the month when a semester begins, September 1st, is clearly a pattern (100 cases) as a new move in date. Being this date so common, let's find which dates are usually suggested as move out date.

SELECT new_move_out_at, COUNT(new_move_out_at) AS date_occurences
FROM booking
WHERE new_move_in_at = '20170901' AND requested_at >= '2017-07-03'
GROUP BY new_move_out_at
ORDER BY date_occurences DESC
LIMIT 10;
R:
	new_move_out_at	date_occurences
	2018-01-31 00:00:00	36
	2018-06-30 00:00:00	10
	2018-08-31 00:00:00	8
	2017-12-30 00:00:00	6
	2017-12-23 00:00:00	4
	2018-07-31 00:00:00	4
	2017-09-30 00:00:00	3
	2018-02-28 00:00:00	3
	2017-11-30 00:00:00	2
	2018-01-27 00:00:00	2

By the results above, when new_move_in_at date is September 1st we can see that in 36 cases the new_move_out_at date is January 31st while in 10 cases its June 30th, which is equivalent to the end of the first and second university semesters, accordingly. The subsequent most common dates are close to the top 2 dates mentioned above. 

The data confirms a seasonal pattern in move in dates at the beginning of the 1st semester (new_move_in_at : 2017-09-01) and at the end of 1st semester (new_move_out_at : 2018-01-31).

Yet, as a landlord, the most important thing may be book full months. This is why we found new_move_in_at date patterns at the first day of the month and new_move_out_at date patterns at the end of the month. The coincidence with semester dates are related with demand. 

Considering all this aquired knowledge, it could be suggested predefined date intervals (full months), taking into account most common demand requests (full semester length or full school year). 


3. Does it influence the acceptance rate of our landlords, i.e. the percentage of bookings accepted?

First of all, is important to say that according to my interpretation and understading of all features and consequent data, 
I decided to use acceptance_at to answer this question, instead of using current_state and its states (e.g. 'accepted').
This being said, in order to analyse the new feature influence in booking's acceptance rate, let's use accepted_at.

Since new feature was implemented in July 3rd, let's see the acceptance rate from this date and without the use of the feature.
SELECT COUNT(accepted_at)
FROM booking
WHERE new_move_in_at IS NULL AND accepted_at >= '2017-07-03';
R: 1795 entries

1795 accepted bookings out of 8353 total entries (21.5%).

Using the feature, the result is
SELECT COUNT(accepted_at)
FROM booking
WHERE new_move_in_at IS NOT NULL AND accepted_at >= '2017-07-03';
R: 312 entries

312 accepted bookings out of 312 new_move_in_at not null cases (100.0%).

Solely by acceptance_at, the acceptance rate improvement was immense, from nearly 22% to a perfect score of 100%. Therefore, this new feature has a massive impact in acceptance rates by landlords.


4. Does it influence the payment rate of our students, i.e. the percentage of bookings paid?

Considering the fact that a student can only pay if the landlord accepted the booking, we will take into consideration accepted_at and current_state.

Doing some exploratory analysis, I realised that a payment can be defined as 'paid' or 'confirmed'

Without using the feature,
SELECT COUNT(*)
FROM booking
WHERE new_move_in_at IS NULL AND requested_at >= '2017-07-03'
AND current_state IN ('paid', 'confirmed');
R: 368 entries

368 bookings were paid, from a total of 8353 (4.4%).

Using the new feature,
SELECT COUNT(*)
FROM booking
WHERE new_move_in_at IS NOT NULL AND requested_at >= '2017-07-03'
AND current_state IN ('paid', 'confirmed');
R: 36 entries

36 bookings out of 312 were paid by students (11.5%).

In conclusion, using this feature, students' payment rate increased from 4.4% to 11.5%.

5. What more data would you require to evaluate the feature and why?

Approaching this problem without prior knowledge, could be useful to have a detailed definition of what each feature mean and how each feature interacts with one another, as there were some doubts related to what each state meant in current_state and how exactly they were related with other features.

Would be interesting to extend, over time, the use of this feature to gather more data, since its a relatively recent (deployed in July 3rd).


2.2
1. What are your main conclusions regarding the results of the feature?

Analysing each figure, one can say that:
- There's an increase of booking requests since the beginning of the test;
- With price adjustments, this feature created interest in published offers that got no booking requests until then (overpriced offers);
- Despite the increase in booking requests, the majority of offers (more than 500) still have no booking requests;
- Offers that suffered a price decrease in the interval of 0 to 100 got more booking requests, although also being the majority of total offers. In cases were the decrease was of 100, the demand even surpassed the supply. 
The fact that offers with higher price decreases didn't had many requests might be related to landlord's high evaluation of its offer. Even with a greater price adjustment it is still not suitable for most people looking for an appartment/room/bed. 
- By "price decrease percent" figure its clear that price adjustments are made accordingly to each offer's price. A price decrease of 20% is predominant although it may not have the greatest success ratio (total requests / total offers).

- Considering "Average click position on search for offers with price recommendation", after July 2017 there was an significant decrease on the average click position per day, which might mean that students found a more interesting offer ranked higher (between 1 and 25). Offers ranked higher as search result, i.e. with lower average click position, became more relevant since July 2017 and maintained the trend during the test, due to a good price recommendation.

- Analysing "Total clicks by search position on offers with price recommendation", better ranked search results (first 20 results) are more relevant than the rest, gathering most of the clicks.

- From "Total clicks per day on offers with price recommendation" we can see see that since the implementation of the feature in 9th of August 2017, offers with price recommendation became more interesting to students and had a great increase in total clicks. 

Summarizing and assessing this test, we can say that price recommendations set by the algorithm in obsolete and "overlooked" offers have a positive influence in creating interest and booking requests. 


2. What other data would you need to evaluate the feature?

It would be interesting to analyse the acceptance rates by landlords regarding booking requests with new prices. Especially in cases of booking rejection, gather information regarding price reduction flexibility. In general, although knowing that this feature created a positive impact in "obsolete" offers, see how much of this new interest resulted in success cases (booked offers).  


3. Can you design an experiment leading to more conclusive results? If so, please describe it.

We can separate results by offer type, analyse price considering offer's characteristics and location (e.g. proximity to university, near public transports, average and variance price values of the area in which the offer is located), gathering average and variance of prices for each offer type. This approach, although may not improve current results, could give a more detailed picture to each offer type and, eventually, help tuning the algorithm. 

Being a fairly recent test, it would be good extend this trial at least a couple of months in order to get a more detailed analysis. The following months (after August) will bring more students to the platform, which may lead to an increase of booking requests in offers with price recommendation.

Another approach would be, considering not booked offers, with similar characteristics and for a certain time threshold (defined by Data Analytics team), could have a new price suggestion (adequate price per offer) taking into account similar booked offers, therefore reducing price discrepancies between "real price" and landlord set overprice. Similar offers can be offers with a certain "membership degree" (high membership degree, with a threshold not too far from 1 if range between 0 and 1) regarding other offers, considering characteristics such as if is property (how many bedrooms), bedroom or bed, its location (e.g. near a specific university, in a particular neighbourhood), etc. Then we could analyse the interest created in those offers by amount of clicks and booking rates, considering rates of "normal" offers and "adapted" offers.
